In [ ]:
# ============================================
# Hue-free freshness modeling pipeline
# ============================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

# === 1. Load HSV data ===
S_wide = pd.read_csv("HSV_S.csv")
V_wide = pd.read_csv("HSV_V.csv")

S_wide.columns = [c.strip().lower() for c in S_wide.columns]
V_wide.columns = [c.strip().lower() for c in V_wide.columns]
id_col = "image_number"
day_cols = [c for c in S_wide.columns if c.startswith("day")]

#| group | condition              |
#| ----- | ---------------------- |
#| 1     | low temp + with water  |
#| 2     | room temp + with water |
#| 3     | low temp + no water    |
#| 4     | room temp + no water   |

group_map = pd.DataFrame({'image_number':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
                           'group':[3,1,2,4,2,3,2,1,3,1,4,3,2,4,4,1]})
S_wide = S_wide.merge(group_map, on='image_number', how='left')

# === 2. Wide → long and merge ===
S_long = S_wide.melt(id_vars=[id_col,"group"], value_vars=day_cols,
                     var_name="day_label", value_name="S")
V_long = V_wide.melt(id_vars=[id_col], value_vars=day_cols,
                     var_name="day_label", value_name="V")

S_long["day"] = S_long["day_label"].str.extract(r"(\d+)").astype(int)
V_long["day"] = V_long["day_label"].str.extract(r"(\d+)").astype(int)
S_long = S_long.drop(columns=["day_label"])
V_long = V_long.drop(columns=["day_label"])


# Keep day1.. only (your files currently have day1..day4)
S_long = S_long.query("day >= 1")
V_long = V_long.query("day >= 1")

df = pd.merge(S_long, V_long, on=[id_col,"day"], how="outer")




df = pd.merge(S_long, V_long, on=[id_col,"day"], how="outer")
# === 3. Map groups to conditions ===
group_map = {
    1: ("low","yes"),
    2: ("room","yes"),
    3: ("low","no"),
    4: ("room","no")
}
df["temp"]  = df["group"].map(lambda g: group_map.get(g,(np.nan,np.nan))[0])
df["water"] = df["group"].map(lambda g: group_map.get(g,(np.nan,np.nan))[1])

# === 4. Normalize and compute freshness ===
df["S"] = pd.to_numeric(df["S"], errors="coerce")


df["V"] = pd.to_numeric(df["V"], errors="coerce")

base = df.groupby(id_col)[["S","V"]].max().rename(columns={"S":"Sbase","V":"Vbase"})
df = df.merge(base, on=id_col, how="left")

df["s"] = ((df["S"])/(df["Sbase"])).clip(1e-6,1.0)
df["v"] = ((df["V"])/(df["Vbase"])).clip(1e-6,1.0)
print(df["s"])
df["F"] = np.sqrt(df["s"] * df["v"])
df["logF"] = np.log(df["F"])

# === 5. Fit mixed-effects model ===
df["temp_code"] = df["temp"].map({"room":0,"low":1})
df["water_code"] = df["water"].map({"no":0,"yes":1})

mdl_df = df.dropna(subset=["logF","day","temp_code","water_code"]).copy()
mdl_df["day"] = mdl_df["day"].astype(float)

model = smf.mixedlm("logF ~ day*water_code*temp_code",
                    mdl_df, groups=mdl_df["image_number"],
                    re_formula="~ day")
fit = model.fit(method="lbfgs")
print(fit.summary())

# === 6. Compute condition-specific slopes  ===
b = fit.params
def slope(temp, water):
    t, w = temp, water
    return (b.get("day",0) +
            b.get("day:water_code",0)*w +
            b.get("day:temp_code",0)*t +
            b.get("day:water_code:temp_code",0)*w*t)

rows = []
for t_name, t_code in [("room",0),("low",1)]:
    for w_name, w_code in [("no",0),("yes",1)]:
        s = slope(t_code,w_code)
        hl = np.log(2)/(-s) if s < 0 else np.nan
        rows.append({"temp":t_name,"water":w_name,"slope_per_day":s})

0     1.000000
1     0.922798
2     0.996003
3     0.892376
4     0.907328
        ...   
75    0.957494
76    0.881360
77    1.000000
78    0.866710
79    0.981742
Name: s, Length: 80, dtype: float64


/usr/local/anaconda/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/anaconda/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/usr/local/anaconda/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 2.130759
  warnings.warn(msg, ConvergenceWarning)
/usr/local/anaconda/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


               Mixed Linear Model Regression Results
Model:                 MixedLM     Dependent Variable:     logF    
No. Observations:      80          Method:                 REML    
No. Groups:            16          Scale:                  0.0006  
Min. group size:       5           Log-Likelihood:         150.8195
Max. group size:       5           Converged:              No      
Mean group size:       5.0                                         
-------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------
Intercept                -0.004    0.016 -0.258 0.796 -0.035  0.027
day                      -0.015    0.004 -3.454 0.001 -0.023 -0.006
water_code               -0.054    0.022 -2.446 0.014 -0.097 -0.011
day:water_code            0.021    0.006  3.406 0.001  0.009  0.033
temp_code                -0.022    0.022 -0.994 0.320 -0.065  0

In [ ]:
# === Pretty plot: individual freshness by group (with consistent color per group) ===
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

def cond_label(t, w):
    temp_name = "cold" if t == "low" else "room"
    water_name = "water" if w == "yes" else "waterless"
    return f"{temp_name} + {water_name}"

df_plot = df.dropna(subset=["F", "day", "temp", "water"]).copy()
df_plot["condition"] = df_plot.apply(lambda r: cond_label(r["temp"], r["water"]), axis=1)

# 1. Assign fixed color per condition
color_map = {
    "cold + water":     "#1f77b4",  # blue
    "cold + waterless": "#9467bd",  # purple
    "room + water":     "#d62728",  # red
    "room + waterless": "#2ca02c",  # green
}

# 2. Style map: same as before
style_map = {
    "cold + water":     {"linestyle": "-",  "marker": "o"},
    "cold + waterless": {"linestyle": "--", "marker": "s"},
    "room + water":     {"linestyle": "-.", "marker": "^"},
    "room + waterless": {"linestyle": ":",  "marker": "d"},
}

# 3. Plot# === Plot individual freshness lines only, same color within each group ===
plt.figure(figsize=(11, 7))

for cond, g in df_plot.groupby("condition"):
    color = color_map[cond]
    for fid, gi in g.groupby("image_number"):
        gi = gi.sort_values("day")
        plt.plot(
            gi["day"], gi["F"],
            linestyle=style_map[cond]["linestyle"],
            marker=style_map[cond]["marker"],
            color=color,
            linewidth=1.6,
            alpha=0.7,          # uniform brightness
            markerfacecolor=color,
            markeredgewidth=0
        )

plt.xlabel("Day", fontsize=12)
plt.ylabel("Freshness score", fontsize=12)
plt.title("Freshness ratings across days", fontsize=14)
plt.xticks(sorted(df_plot["day"].unique()))
plt.grid(True, alpha=0.25)

# Start from day 1 and fit y-axis
plt.xlim(1, df_plot["day"].max() + 0.2)
plt.ylim(df_plot["F"].min() - 0.02, df_plot["F"].max() + 0.02)

# Legend (one per condition)
handles = [
    Line2D([0], [0],
           linestyle=style_map[k]["linestyle"],
           marker=style_map[k]["marker"],
           color=color_map[k],
           linewidth=2.5, label=k)
    for k in style_map
]
plt.legend(handles=handles, title="Condition", ncol=2, frameon=True)

plt.tight_layout()
plt.savefig("freshness_by_group_individuals_only.png", dpi=200)
plt.close()

print("Saved: freshness_by_group_individuals_only.png")


Saved: freshness_by_group_individuals_only.png


In [38]:
# ============================================
# Permutation test on day1–day5 freshness change
# ============================================
import pandas as pd
import numpy as np

# --- 1) Load HSV data (already has day1–day5) ---
S_wide = pd.read_csv("HSV_S.csv")
V_wide = pd.read_csv("HSV_V.csv")

S_wide.columns = [c.strip().lower() for c in S_wide.columns]
V_wide.columns = [c.strip().lower() for c in V_wide.columns]
id_col = "image_number"
day_cols = [c for c in S_wide.columns if c.startswith("day")]

# --- 2) Randomization mapping (same as before) ---
group_assign = pd.DataFrame({
    'image_number':[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16],
    'group':[3,1,2,4,2,3,2,1,3,1,4,3,2,4,4,1]
})
S_wide = S_wide.merge(group_assign, on="image_number", how="left")

# --- 3) Wide → long merge ---
S_long = S_wide.melt(id_vars=[id_col,"group"], value_vars=day_cols,
                     var_name="day_label", value_name="S")
V_long = V_wide.melt(id_vars=[id_col], value_vars=day_cols,
                     var_name="day_label", value_name="V")
S_long["day"] = S_long["day_label"].str.extract(r"(\d+)").astype(int)
V_long["day"] = V_long["day_label"].str.extract(r"(\d+)").astype(int)
S_long = S_long.drop(columns=["day_label"])
V_long = V_long.drop(columns=["day_label"])
df = pd.merge(S_long, V_long, on=[id_col,"day"], how="outer")

# --- 4) Map groups to (temp, water) ---
group_map = {
    1: ("low","yes"),   # low temp + water
    2: ("room","yes"),  # room temp + water
    3: ("low","no"),    # low temp + no water
    4: ("room","no")    # room temp + no water
}
df["temp"]  = df["group"].map(lambda g: group_map[g][0])
df["water"] = df["group"].map(lambda g: group_map[g][1])

# --- 5) Compute freshness F ---
df["S"] = pd.to_numeric(df["S"], errors="coerce")
df["V"] = pd.to_numeric(df["V"], errors="coerce")

base = df.groupby(id_col)[["S","V"]].max().rename(columns={"S":"Sbase","V":"Vbase"})
df = df.merge(base, on=id_col, how="left")
df["s"] = (df["S"]/df["Sbase"]).clip(1e-6,1.0)
df["v"] = (df["V"]/df["Vbase"]).clip(1e-6,1.0)
df["F"] = np.sqrt(df["s"] * df["v"])

# --- 6) ΔF = F(day5) − F(day1) per flower ---
df["day"] = df["day"].astype(int)
delta = (
    df.pivot_table(index="image_number", columns="day", values="F")
    .assign(diff=lambda x: x[5] - x[1])
    .reset_index()[["image_number","diff"]]
)
delta = delta.merge(group_assign, on="image_number", how="left")
delta["temp"]  = delta["group"].map(lambda g: group_map[g][0])
delta["water"] = delta["group"].map(lambda g: group_map[g][1])

# =======================================================
#  (A) Water effect: within each temperature
# =======================================================
obs_water = {}
perm_pvals_water = {}
n_perm = 10000

for temp in ["low","room"]:
    sub = delta[delta["temp"]==temp]
    obs_diff = sub[sub["water"]=="yes"]["diff"].mean() - sub[sub["water"]=="no"]["diff"].mean()
    obs_water[temp] = obs_diff

    diffs = []
    for _ in range(n_perm):
        shuffled = sub.copy()
        shuffled["water"] = np.random.permutation(shuffled["water"].values)
        m_yes = shuffled[shuffled["water"]=="yes"]["diff"].mean()
        m_no  = shuffled[shuffled["water"]=="no"]["diff"].mean()
        diffs.append(m_yes - m_no)
    diffs = np.array(diffs)
    pval = (np.sum(diffs >= obs_diff) + 1) / (len(diffs) + 1)
    perm_pvals_water[temp] = pval

# =======================================================
#  (B) Temperature effect: within each water condition
# =======================================================
obs_temp = {}
perm_pvals_temp = {}

for water in ["yes","no"]:
    sub = delta[delta["water"]==water]
    obs_diff = sub[sub["temp"]=="low"]["diff"].mean() - sub[sub["temp"]=="room"]["diff"].mean()
    obs_temp[water] = obs_diff

    diffs = []
    for _ in range(n_perm):
        shuffled = sub.copy()
        shuffled["temp"] = np.random.permutation(shuffled["temp"].values)
        m_low  = shuffled[shuffled["temp"]=="low"]["diff"].mean()
        m_room = shuffled[shuffled["temp"]=="room"]["diff"].mean()
        diffs.append(m_low - m_room)
    diffs = np.array(diffs)
    pval = (np.sum(diffs >= obs_diff) + 1) / (len(diffs) + 1)
    perm_pvals_temp[water] = pval

# =======================================================
#  Combine & report
# =======================================================
min_p = min(list(perm_pvals_water.values()) + list(perm_pvals_temp.values()))

print("\nObserved water effects (with - no water):", obs_water)
print("Permutation p-values (water effects):", perm_pvals_water)
print("\nObserved temperature effects (low - room):", obs_temp)
print("Permutation p-values (temperature effects):", perm_pvals_temp)
print("\nMinimum empirical p-value across all tests =", min_p)



Observed water effects (with - no water): {'low': 0.04227971933573854, 'room': 0.06101556208621067}
Permutation p-values (water effects): {'low': 0.0273972602739726, 'room': 0.011998800119988001}

Observed temperature effects (low - room): {'yes': -0.0006063337921292933, 'no': 0.018129508958342838}
Permutation p-values (temperature effects): {'yes': 0.5278472152784721, 'no': 0.15958404159584041}

Minimum empirical p-value across all tests = 0.011998800119988001
